# Analyze runtimes

Analyze the runtimes of the injections we did

## Preamble

In [172]:
import psutil
p = psutil.Process()
p.cpu_affinity([0])
import os
import json
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np 
import matplotlib.pyplot as plt
from IPython.display import Latex

params = {"axes.grid": True,
        "text.usetex" : True,
        "font.family" : "serif",
        "ytick.color" : "black",
        "xtick.color" : "black",
        "axes.labelcolor" : "black",
        "axes.edgecolor" : "black",
        "font.serif" : ["Computer Modern Serif"],
        "xtick.labelsize": 16,
        "ytick.labelsize": 16,
        "axes.labelsize": 16,
        "legend.fontsize": 16,
        "legend.title_fontsize": 16,
        "figure.titlesize": 16}

plt.rcParams.update(params)
import copy

In [173]:
outdirs_injections = {"TF2": "../injections/outdir_TF2/",
                      "NRTv2": "../injections/outdir_NRTv2/"}

jim_paths = {"GW170817_TaylorF2": "../real_events/GW170817_TaylorF2/outdir/",
             "GW170817_NRTidalv2": "../real_events/GW170817_NRTidalv2/outdir/",
             "GW190425_TaylorF2": "../real_events/GW190425_TaylorF2/outdir/",
             "GW190425_NRTidalv2": "../real_events/GW190425_NRTidalv2/outdir/"}

bilby_paths = {"GW170817_TaylorF2": "/home/thibeau.wouters/jim_pbilby_samples/GW170817/GW170817-TF2_rejection_sampling_result.json",
               "GW170817_NRTidalv2": "/home/thibeau.wouters/jim_pbilby_samples/GW170817/GW170817_IMRDNRTv2_older_bilby_result.json",
               "GW190425_TaylorF2": "/home/thibeau.wouters/jim_pbilby_samples/GW190425/GW190425-TF2_result.json",
               "GW190425_NRTidalv2": "/home/thibeau.wouters/jim_pbilby_samples/GW190425/GW190425_IMRDNRTv2_older_bilby_result.json"}

run_names = list(jim_paths.keys())

In [174]:
def format_runtime(runtime_minutes: float, 
                   nb_round: int = 2) -> str:
    """
    Simple function to format the runtime in minutes appropriately.

    Args:
        runtime_minutes (float): Runtime in minutes
        nb_round (int, optional): Number of digits for rounding. Defaults to 2.

    Returns:
        str: Nicely formatted runtime
    """
    runtime_string = str(np.round(runtime_minutes, nb_round))
    if len(runtime_string) == 4:
        runtime_string += "0"
        
    return runtime_string

In [175]:
# def fetch_runtime_jim(event_name: str, 
#                       outdir_name: str,
#                       nb_round: int = 2) -> tuple[float, str]:
#     """
#     Fetch and format the runtime of analyzing one of the real events 

#     Args:
#         event_name (str): Identifier of the event, format GW_WF, e.g. GW170817_TaylorF2
#         outdir_name (str): Name of the outdir, e.g. `outdir`
#         runtime_filename (str, optional): _description_. Defaults to "runtime.txt".
#         nb_round (int, optional): _description_. Defaults to 2.

#     Returns:
#         tuple: Tuple of float value, being the runtime in seconds, and a string with the runtime in minutes
#     """
#     # path = f"../real_events/{event_name}/{outdir_name}/{runtime_filename}"
#     path = f"../real_events/{event_name}/{outdir_name}/"
#     try:
#         ext_list = ["runtime_training.txt", "runtime_production.txt"]
#         runtimes = [np.loadtxt(path + ext) for ext in ext_list]
#     except Exception as e:
#         print(f"Error: {e}")
#         ext_list = ["runtime.txt"]
#         runtimes = [np.loadtxt(path + ext) for ext in ext_list]
    
#     runtime_seconds = np.sum(runtimes)
#     runtime_minutes = runtime_seconds / 60
#     runtime_string = format_runtime(runtime_minutes, nb_round)
    
#     return runtime_seconds, runtime_string

In [176]:
def fetch_runtime_jim(run_dir: str,
                      nb_round: int = 2) -> float:
    """
    Fetch the runtime of a given run directory of a jim run.

    Args:
        run_dir (str): Directory of the run, i.e. the outdir

    Returns:
        float: Runtime.
    """
    
    # In case runtime for training and production are saved separately:
    if os.path.exists(run_dir + "/runtime_training.txt") and os.path.exists(run_dir + "/runtime_production.txt"):
        with open(run_dir + "/runtime_training.txt", "r") as f:
            runtime_training = float(f.read())
            
        with open(run_dir + "/runtime_production.txt", "r") as f:
            runtime_production = float(f.read())
        
        runtime_seconds = runtime_training + runtime_production
        
    else:
        with open(run_dir + "/runtime.txt", "r") as f:
            runtime_seconds = float(f.read())
            
    runtime_minutes = runtime_seconds / 60
    runtime_string = format_runtime(runtime_minutes, nb_round)
            
    return runtime_seconds, runtime_string

In [177]:
def fetch_runtime_bilby(event_name: str, 
                        nb_round: int = 2,
                        convert_to_hours: bool = True) -> tuple[float, str]:
    """
    Fetch and format the runtime of analyzing one of the real events for bilby. 

    Args:
        event_name (str): Identifier of the event, format GW_WF, e.g. GW170817_TaylorF2
        outdir_name (str): Name of the outdir, e.g. `outdir`
        runtime_filename (str, optional): _description_. Defaults to "runtime.txt".
        nb_round (int, optional): _description_. Defaults to 2.

    Returns:
        tuple: Tuple of float value, being the runtime in seconds, and a string with the runtime in minutes
    """
    if event_name not in bilby_paths.keys():
        raise ValueError(f"Event {event_name} not in the list of events for which we have bilby runtimes")
    
    path = bilby_paths[event_name]
    with open(path, "r") as f:
        bilby_data = json.load(f)
        runtime_seconds = float(bilby_data["sampling_time"])
        
    runtime_minutes = runtime_seconds / 60
    if convert_to_hours:
        runtime_minutes /= 60 # TODO fix the naming, or not, if I am too lazy in the end
    runtime_string = format_runtime(runtime_minutes, nb_round)
    
    return runtime_seconds, runtime_string

## Injections runtimes

In [178]:
runtimes_dict = {} 

for name in outdirs_injections.keys():
    outdir = outdirs_injections[name]
    runtimes = []
    for subdir in os.listdir(outdir):
        rundir = outdir + subdir
        runtime_path = rundir + "/runtime.txt"
        
        try:
            runtime, _ = fetch_runtime_jim(rundir)
            runtimes.append(runtime)
        except Exception as e:
            print(f"Skipping {rundir}, error: {e}")
            
    runtimes_dict[name] = np.array(runtimes)

Skipping ../injections/outdir_TF2/injection_54, error: [Errno 2] No such file or directory: '../injections/outdir_TF2/injection_54/runtime.txt'


Make a histogram of the runtimes, more as a sanity check for myself.

In [179]:
minutes = True # whether to convert runtimes to minutes rather than seconds

if minutes:
    denominator = 60
else:
    denominator = 1

hist_TF2, edges = np.histogram(runtimes_dict["TF2"] / denominator, bins = 20, density = True)
hist_NRTv2, _ = np.histogram(runtimes_dict["NRTv2"] / denominator, bins = edges, density = True)

if minutes:
    xlabel = "Runtime (min)"
else:
    xlabel = "Runtime (s)"

stairs_kwargs = {"linewidth": 2}

# Create the figure
plt.figure(figsize=(10, 6))
plt.stairs(hist_TF2, edges, label = "TF2", **stairs_kwargs)
plt.stairs(hist_NRTv2, edges, label = "NRTv2", **stairs_kwargs)

plt.legend()
plt.xlabel(xlabel)
plt.ylabel("Density")
# plt.show()
plt.close()

## Runtimes for compilation and initializing the likelihood

In [180]:
runtimes_compilation_filename = "../data/runtimes_compilation.txt"
runtimes_compilation = np.loadtxt(runtimes_compilation_filename, delimiter=",")

compilation_mean, compilation_std = np.mean(runtimes_compilation), np.std(runtimes_compilation)
print(f"Compilation: mean +/- std: {compilation_mean / 60:.2f} +/- {compilation_std / 60:.2f} minutes")

compilation_mean_minutes = compilation_mean / 60

Compilation: mean +/- std: 2.59 +/- 0.33 minutes


## Final numbers

Can subtract the time for compiling etc, if wanted:

In [181]:
jim_runtimes_injections = {}
jim_runtimes_injections_median = {}
subtract_number = compilation_mean_minutes

for name, runtimes in runtimes_dict.items():
    
    ### Mean +/- std:
    mean_str = format_runtime(np.mean(runtimes / denominator) - subtract_number)
    std_str = format_runtime(np.std(runtimes / denominator))
    median_str = format_runtime(np.median(runtimes / denominator) - subtract_number)
    
    jim_runtime_str = r"$({} \pm {})$ min".format(mean_str, std_str)
    print(f"Mean and std for {name}")
    print(jim_runtime_str)
    jim_runtimes_injections[name] = jim_runtime_str
    
    ### Median:
    jim_runtime_str = r"${}$ min".format(median_str)
    print(f"Median for {name}")
    print(jim_runtime_str)
    
    jim_runtimes_injections_median[name] = jim_runtime_str

Mean and std for TF2
$(24.49 \pm 12.18)$ min
Median for TF2
$22.17$ min
Mean and std for NRTv2
$(18.51 \pm 9.160)$ min
Median for NRTv2
$15.43$ min


## Fetch jim runtimes

In [182]:
jim_runtimes_float = {}
jim_runtimes_str = {}

for run in run_names:
    print(run)
    path = jim_paths[run]
    print(path)
    runtime_float, runtime_str = fetch_runtime_jim(path)
    jim_runtimes_float[run] = runtime_float
    jim_runtimes_str[run] = runtime_str

GW170817_TaylorF2
../real_events/GW170817_TaylorF2/outdir/
GW170817_NRTidalv2
../real_events/GW170817_NRTidalv2/outdir/
GW190425_TaylorF2
../real_events/GW190425_TaylorF2/outdir/
GW190425_NRTidalv2
../real_events/GW190425_NRTidalv2/outdir/


## Fetch pbilby runtimes

In [183]:
# Override with the numbers obtained from the log file of the runs instead:
bilby_runtimes_total = {}
bilby_runtimes_total["GW170817_TaylorF2"]  = 0.0
bilby_runtimes_total["GW170817_NRTidalv2"] = 12 * (60) ** 2 + 34 * 60 + 15
bilby_runtimes_total["GW190425_TaylorF2"]  = 0.0
bilby_runtimes_total["GW190425_NRTidalv2"] =  5 * (60) ** 2 + 39 * 60 + 38

In [184]:
bilby_runtimes_float = {}
bilby_runtimes_str = {}
for key, path in bilby_paths.items():
    runtime_float = bilby_runtimes_total[key]
    runtime_minutes = runtime_float / 60
    runtime_string = format_runtime(runtime_minutes, 2)
        
    bilby_runtimes_float[key] = runtime_float
    bilby_runtimes_str[key] = runtime_str

In [185]:
# Add dollar signs and minute string
for key, value in jim_runtimes_str.items():
    jim_runtimes_str[key] = f"${value}$ min"

In [186]:
assert list(jim_runtimes_float.keys()) == list(bilby_runtimes_float.keys())
assert list(jim_runtimes_str.keys()) == list(bilby_runtimes_str.keys())

Put hours in the formatting for pbilby

In [187]:
for key, value in bilby_runtimes_str.items():
    bilby_runtimes_str[key] = f"${value}$ h"

## Get the ratio of runtimes

In [188]:
runtimes_ratio_float = {}
runtimes_ratio_str = {}

for key in jim_runtimes_float.keys():
    ratio = bilby_runtimes_float[key] / jim_runtimes_float[key]
    runtimes_ratio_float[key] = ratio
    print(ratio)
    runtimes_ratio_str[key] = f"${int(ratio)} \\times $"

0.0
39.39925638160177
0.0
17.423311104088803


## Combine into table

In [189]:
latex_code = '\\begin{{tabular}}{{l l c c c}}\n Event & Waveform & \\texttt{{jim}} & \\texttt{{pbilby}} (480 cores) & Speedup \\\\\n \hline\\hline\n\\multirow{{2}}{{*}}{{GW170817}} & TF2 & {} & {} & {} \\\\\n & NRTv2 & {} & {} & {} \\\\ \hline\n\\multirow{{2}}{{*}}{{GW190425}}  & TF2 & {} & {} & {} \\\\ \n & NRTv2 & {} & {} & {} \\\\ \hline\n\\multirow{{2}}{{*}}{{Injection}} & TF2 & {} & -- & -- \\\\\n& NRTv2 & {} & -- & -- \\\\\n\\hline\\hline\n\\end{{tabular}}'\
.format(  jim_runtimes_str["GW170817_TaylorF2"],
        bilby_runtimes_str["GW170817_TaylorF2"],
        runtimes_ratio_str["GW170817_TaylorF2"],
          jim_runtimes_str["GW170817_NRTidalv2"],
        bilby_runtimes_str["GW170817_NRTidalv2"], 
        runtimes_ratio_str["GW170817_NRTidalv2"],
          jim_runtimes_str["GW190425_TaylorF2"], 
        bilby_runtimes_str["GW190425_TaylorF2"], 
        runtimes_ratio_str["GW190425_TaylorF2"],
          jim_runtimes_str["GW190425_NRTidalv2"], 
        bilby_runtimes_str["GW190425_NRTidalv2"],
        runtimes_ratio_str["GW190425_NRTidalv2"], 
        jim_runtimes_injections["TF2"], 
        jim_runtimes_injections["NRTv2"]
       )

print(latex_code)

\begin{tabular}{l l c c c}
 Event & Waveform & \texttt{jim} & \texttt{pbilby} (480 cores) & Speedup \\
 \hline\hline
\multirow{2}{*}{GW170817} & TF2 & $25.72$ min & $19.49$ h & $0 \times $ \\
 & NRTv2 & $19.14$ min & $19.49$ h & $39 \times $ \\ \hline
\multirow{2}{*}{GW190425}  & TF2 & $45.76$ min & $19.49$ h & $0 \times $ \\ 
 & NRTv2 & $19.49$ min & $19.49$ h & $17 \times $ \\ \hline
\multirow{2}{*}{Injection} & TF2 & $(24.49 \pm 12.18)$ min & -- & -- \\
& NRTv2 & $(18.51 \pm 9.160)$ min & -- & -- \\
\hline\hline
\end{tabular}


## Electricity consumption

[Source](https://ark.intel.com/content/www/us/en/ark/products/136874/intel-xeon-platinum-8174-processor-33m-cache-3-10-ghz.html) for cluster

Conversions: taken from the [following webpage](https://git.ligo.org/lscsoft/parallel_bilby/-/wikis/O4-Review#pbilby-201-202):

"For the capture of 1 tonne of CO2 emissions ~50 trees must grow for one year [1]
^using 0.905 kg of CO2 per kilowatt hour [2], and 0.3kWh/hr per CPU."

[1] https://www.climateneutralgroup.com/en/news/what-exactly-is-1-tonne-of-co2-v2/

[2] https://www.dcceew.gov.au/climate-change/publications/national-greenhouse-gas-inventory-quarterly-updates

In [190]:
kwh_to_co2 = 0.905 # kg CO2 per kWh
co2_to_trees = (1 / 1000) * 50 # trees per kg CO2, growing for one year

consumption_cpu = 0.240 # kilowatts
consumption_gpu = 0.400 # kilowatts
N_cores = 480
N_runs = 204

In [191]:
consumption_jim = {}
for key in jim_runtimes_float.keys():
    consumption_jim[key] = 1 * consumption_cpu * (jim_runtimes_float[key] / 3600)
    
for key, value in consumption_jim.items():
    print(f"Consumption for {key}: {value:.2f} kWh")

Consumption for GW170817_TaylorF2: 0.10 kWh
Consumption for GW170817_NRTidalv2: 0.08 kWh
Consumption for GW190425_TaylorF2: 0.18 kWh
Consumption for GW190425_NRTidalv2: 0.08 kWh


In [192]:
consumption_bilby = {}
for key in bilby_runtimes_float.keys():
    consumption_bilby[key] = N_cores * consumption_cpu * (bilby_runtimes_float[key] / 3600)
    
for key, value in consumption_bilby.items():
    print(f"Consumption for {key}: {value:.2f} kWh")

Consumption for GW170817_TaylorF2: 0.00 kWh
Consumption for GW170817_NRTidalv2: 1448.16 kWh
Consumption for GW190425_TaylorF2: 0.00 kWh
Consumption for GW190425_NRTidalv2: 652.10 kWh


In [193]:
consumption_ratio = {}
for key in consumption_jim.keys():
    consumption_ratio[key] = int(np.round(consumption_bilby[key] / consumption_jim[key]))
    
for key, value in consumption_ratio.items():
    print(f"Consumption for {key}: {value} x")

Consumption for GW170817_TaylorF2: 0 x
Consumption for GW170817_NRTidalv2: 18912 x
Consumption for GW190425_TaylorF2: 0 x
Consumption for GW190425_NRTidalv2: 8363 x


Convert to trees now

In [194]:
def get_number_of_killed_trees(kwh: float) -> float:
    """
    Get the number of trees killed by the energy consumption of our runs in kWh.

    Args:
        kwh (float): Energy consumption in kWh

    Returns:
        float: Number of trees killed
    """
    return kwh * kwh_to_co2 * co2_to_trees

In [195]:
trees_jim = {}
for key, value in consumption_jim.items():
    trees_jim[key] = get_number_of_killed_trees(value)
    
for key, value in trees_jim.items():
    print(f"Trees killed for {key}: {value:.5f}")
    
# Find the average:
avg_nb_killed_trees_jim = np.mean(list(trees_jim.values()))

Trees killed for GW170817_TaylorF2: 0.00465
Trees killed for GW170817_NRTidalv2: 0.00347
Trees killed for GW190425_TaylorF2: 0.00828
Trees killed for GW190425_NRTidalv2: 0.00353


In [196]:
trees_bilby = {}
for key, value in consumption_bilby.items():
    trees_bilby[key] = get_number_of_killed_trees(value)
    
for key, value in trees_bilby.items():
    print(f"Trees killed for {key}: {value:.2f}")

Trees killed for GW170817_TaylorF2: 0.00
Trees killed for GW170817_NRTidalv2: 65.53
Trees killed for GW190425_TaylorF2: 0.00
Trees killed for GW190425_NRTidalv2: 29.51


For a comparison about the whole paper, just use this average and the number of runs done to allow for an easier comparison between the 2:

In [197]:
avg_nb_killed_trees_jim = np.mean(list(trees_jim.values()))
avg_nb_killed_trees_bilby = np.mean(list(trees_bilby.values()))

print(f"Average number of killed trees for writing the paper with   jim: {(N_runs * avg_nb_killed_trees_jim):.2f}")
print(f"Average number of killed trees for writing the paper with bilby: {(N_runs * avg_nb_killed_trees_bilby):.2f}")

Average number of killed trees for writing the paper with   jim: 1.02
Average number of killed trees for writing the paper with bilby: 4846.87
